In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
def GECToR(sentence):
    model_path = '/content/drive/MyDrive/ML_project/fine_tuned_gec_model'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

    inputs = tokenizer(sentence, return_tensors='pt', max_length=64, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs)
    corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_sentence

test = "She will goes to the market yesterday but forgot her wallet."
print('original : ' + test)
print('correct : ' + GECToR(test))


original : She will goes to the market yesterday but forgot her wallet.
correct : She went to the market yesterday but forgot her wallet.


In [5]:
def coedit(sentence):
  model_path = '/content/drive/MyDrive/ML_project/coedit'
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  tokenizer = T5Tokenizer.from_pretrained(model_path, legacy=False, local_files_only=True)
  model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True).to('cpu')
  input_text = f"fix grammar: {sentence}"
  inputs = tokenizer(input_text, return_tensors='pt', max_length=64, truncation=True).to(device)
  with torch.no_grad():
      outputs = model.generate(**inputs, max_length=64, num_beams=5, early_stopping=True)
  corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return corrected_sentence

test = "She will goes to the market yesterday but forgot her wallet."
print('original : ' + test)
print('correct : ' + coedit(test))



original : She will goes to the market yesterday but forgot her wallet.
correct : She went to the market yesterday but forgot her wallet.


In [6]:
def BART(sentence):
  model_path = '/content/drive/MyDrive/ML_project/final_model'
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  tokenizer = T5Tokenizer.from_pretrained(model_path, legacy=False, local_files_only=True)
  model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True).to('cpu')
  input_text = f"fix grammar: {sentence}"
  inputs = tokenizer(input_text, return_tensors='pt', max_length=64, truncation=True).to(device)
  with torch.no_grad():
      outputs = model.generate(**inputs, max_length=64, num_beams=5, early_stopping=True)
  corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return corrected_sentence

test = "She will goes to the market yesterday but forgot her wallet."
print('original : ' + test)
print('correct : ' + BART(test))

original : She will goes to the market yesterday but forgot her wallet.
correct : She went to the market yesterday, but forgot her wallet.


In [7]:
def Classification(sentence):
  model_path = '/content/drive/MyDrive/ML_project/Classification'
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
  model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True).to('cpu')
  inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=64).to(device)
  with torch.no_grad():
      outputs = model(**inputs)
  score = torch.softmax(outputs.logits, dim=-1)[0][1].item()
  return score

test = "She will goes to the market yesterday but forgot her wallet."
print('original : ' + test)
print(f'correct : {Classification(test)}') # Use f-string for formatting

original : She will goes to the market yesterday but forgot her wallet.
correct : 0.004988500382751226


In [ ]:
test = input("Enter a sentence:")
def my_NN(sentence):
  first_1 = GECToR(coedit(BART(sentence)))
  first_2 = GECToR(BART(coedit(sentence)))
  second_1 = coedit(BART(GECToR(sentence)))
  second_2 = coedit(GECToR(BART(sentence)))
  third_1 = BART(GECToR(coedit(sentence)))
  third_2 = BART(coedit(GECToR(sentence)))
  l1 = []
  l1.append(first_1)
  l1.append(first_2)
  l1.append(second_1)
  l1.append(second_2)
  l1.append(third_1)
  l1.append(third_2)
  score = []
  for i in l1:
    score.append(Classification(i))
  return l1[score.index(max(score))]

print('original : ' + test)
print('correct : ' + my_NN(test))




In [ ]:
test_cases = [
    ("She will goes to the market yesterday but forgot her wallet.", "She went to the market yesterday, but forgot her wallet."),
    ("Me and him goes to store and buyed candies.", "He and I went to the store and bought candies."),
    ("He don't knows nothing about fixing cars.", "He doesn't know anything about fixing cars."),
    ("Although he was tired. Decided to keep working.", "Although he was tired, he decided to keep working."),
    ("Their going too the park later, isn't it?", "They're going to the park later, aren't they?"),
    ("The dog bark loudly every morning.", "The dog barks loudly every morning."),
    ("He didn't knew about the meeting.", "He didn't know about the meeting."),
    ("Me and her went to the cinema.", "She and I went to the cinema."),
    ("She don't wants to go out tonight.", "She doesn't want to go out tonight."),
    ("We was watching a movie yesterday.", "We were watching a movie yesterday."),
    ("I didn't see nobody at the party.", "I didn't see anybody at the party."),
    ("She is good in math.", "She is good at math."),
    ("The flowers smells nice.", "The flowers smell nice."),
    ("Running fast, the finish line was reached.", "Running fast, he reached the finish line."),
    ("She buyed a new dress last week.", "She bought a new dress last week."),
    ("Them is going to the event.", "They are going to the event."),
    ("He was so tired, he slept all the day.", "He was so tired that he slept all day."),
    ("I seen him at the mall yesterday.", "I saw him at the mall yesterday."),
    ("She enjoy to read books.", "She enjoys reading books."),
    ("Her don't like spicy food.", "She doesn't like spicy food."),
    ("They was playing soccer when I arrived.", "They were playing soccer when I arrived."),
    ("Him is taller than me.", "He is taller than me."),
    ("The book were on the table.", "The book was on the table."),
    ("He don't have no money.", "He doesn't have any money."),
    ("The cat was sitting on a tree who was very high.", "The cat was sitting on a tree that was very high."),
    ("She is married with a lawyer.", "She is married to a lawyer."),
    ("He doesn't knows where the library is.", "He doesn't know where the library is."),
    ("While walking through the park, the birds sang beautifully.", "While walking through the park, he heard the birds singing beautifully."),
    ("Me and my friend likes pizza.", "My friend and I like pizza."),
    ("He was interested on the new project.", "He was interested in the new project."),
    ("I cannot able to do it.", "I cannot do it."),
    ("She is knowing the answer.", "She knows the answer."),
    ("If I was you, I would apologize.", "If I were you, I would apologize."),
    ("The movie was too much long.", "The movie was too long."),
    ("I am agree with your opinion.", "I agree with your opinion."),
    ("She said me to come early.", "She told me to come early."),
    ("The train leave at 5 PM every day.", "The train leaves at 5 PM every day."),
    ("I have seen her yesterday.", "I saw her yesterday."),
    ("He suggested me to join the team.", "He suggested that I join the team."),
    ("They was not ready for the trip.", "They were not ready for the trip."),
    ("Neither of the boys are responsible.", "Neither of the boys is responsible."),
    ("She sings good.", "She sings well."),
    ("He is very interesting in science.", "He is very interested in science."),
    ("Me went to the party alone.", "I went to the party alone."),
    ("He didn't knew what to say.", "He didn't know what to say."),
    ("The team are playing well.", "The team is playing well."),
    ("You should to try harder.", "You should try harder."),
    ("He is more taller than me.", "He is taller than me."),
    ("She do her homework every evening.", "She does her homework every evening."),
    ("Each of the students were given a book.", "Each of the students was given a book."),
    ("He doesn't likes coffee.", "He doesn't like coffee."),
    ("This place is more better than the other one.", "This place is better than the other one."),
    ("I wish I was there.", "I wish I were there."),
    ("He has went to the market.", "He has gone to the market."),
    ("The dog is barking on the stranger.", "The dog is barking at the stranger."),
    ("He explained me the problem.", "He explained the problem to me."),
    ("She hasn't no time to talk.", "She has no time to talk."),
    ("I am looking forward to meet you.", "I am looking forward to meeting you."),
    ("She can sings beautifully.", "She can sing beautifully."),
    ("He don't likes working on weekends.", "He doesn't like working on weekends."),
    ("They didn't had lunch yet.", "They haven't had lunch yet."),
    ("He is very tired, but he keep working.", "He is very tired, but he keeps working."),
    ("This is the house where I was born in.", "This is the house where I was born."),
    ("He suggested me that I should join.", "He suggested that I should join."),
    ("She do not wants to leave.", "She does not want to leave.")
]
# i want to remove the second element of each tuble
for i in range(len(test_cases)):
    test_cases[i] = test_cases[i][0]
print(test_cases)

['She will goes to the market yesterday but forgot her wallet.', 'Me and him goes to store and buyed candies.', "He don't knows nothing about fixing cars.", 'Although he was tired. Decided to keep working.', "Their going too the park later, isn't it?", 'The dog bark loudly every morning.', "He didn't knew about the meeting.", 'Me and her went to the cinema.', "She don't wants to go out tonight.", 'We was watching a movie yesterday.', "I didn't see nobody at the party.", 'She is good in math.', 'The flowers smells nice.', 'Running fast, the finish line was reached.', 'She buyed a new dress last week.', 'Them is going to the event.', 'He was so tired, he slept all the day.', 'I seen him at the mall yesterday.', 'She enjoy to read books.', "Her don't like spicy food.", 'They was playing soccer when I arrived.', 'Him is taller than me.', 'The book were on the table.', "He don't have no money.", 'The cat was sitting on a tree who was very high.', 'She is married with a lawyer.', "He doesn't

In [ ]:
for i in test_cases:
  print('original : ' + i)
  print('correct : ' + my_NN(i))

original : She will goes to the market yesterday but forgot her wallet.
correct : She went to the market yesterday, but forgot her wallet.
original : Me and him goes to store and buyed candies.
correct : He and I went to the store and bought some candies.
original : He don't knows nothing about fixing cars.
correct : He doesn't know anything about fixing cars.
original : Although he was tired. Decided to keep working.
correct : Although he was tired, he decided to keep working.
original : Their going too the park later, isn't it?
correct : They're going to the park later, though.
original : The dog bark loudly every morning.
correct : The dog barks loudly every morning.
original : He didn't knew about the meeting.
correct : He didn't know about the meeting.
original : Me and her went to the cinema.
correct : She and I went to the cinema.
original : She don't wants to go out tonight.
correct : She doesn't want to go out tonight.
original : We was watching a movie yesterday.
correct : We